# Decision Trees for detecting heart disease

In this lesson, we will train a simple binary decision tree.

## Import modules

In [1]:
import pandas as pd             # to load and manipulate data
import numpy as np              # to manipulate with vectors and matrices

## Import and preprocess data

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data', 
                  header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


By information on the dataset page https://archive.ics.uci.edu/ml/datasets/Heart+Disease we have the following variables: 
- **age**,
- **sex**, 1 -- male, 0 -- female;
- **cp**, chest pain type: 
    - Value 1: typical angina, 
    - Value 2: atypical angina
    - Value 3: non-anginal pain
    - Value 4: asymptomatic 
- **restbp**, resting blood pressure (in mm Hg)
- **chol**, serum cholesterol in mg/dl
- **fbs**, fasting blood sugar
- **restecg**, resting electrocardiographic results
- **thalach**,  maximum heart rate achieved
- **exang**, exercise induced angina
- **oldpeak**, ST depression induced by exercise relative to rest
- **slope**, the slope of the peak exercise ST segment.
- **ca**, number of major vessels (0-3) colored by fluoroscopy
- **thal**, this is short of thalium heart scan.
- **hd**, diagnosis of heart disease, the predicted attribute: 0 for no diagnozed heart disease.

The good idea is to rename columns in this way.

In [3]:
## change the column numbers to column names
df.columns = ['age', 
              'sex', 
              'cp', 
              'restbp', 
              'chol', 
              'fbs', 
              'restecg', 
              'thalach', 
              'exang', 
              'oldpeak', 
              'slope', 
              'ca', 
              'thal', 
              'hd']

Our goal is to distinguish presence (values 1,2,3,4) from absence (value 0) of heart disease, that corresponds to the value of the last column. It will be our **target** variable.

To make a simple model, we do not require to use all the data. To simplify, we suggest the following binary variables of our interst:
* age is less or equal than 40;
* age is greater than 60;
* sex is male (0 corresonds to female);
* chest pain is not asymptomatic (in other words, patient has a chest pain);
* is fasting blood sugar level to high;
* resting electrocardiographic results are normal;
* has exercise induced angina;
* resting blood pressure greater or equal than 130 (corresponds to hypertension);

In [4]:
X = np.array([
    df['age'] <= 40, 
    df['age'] > 60, 
    df['sex'], 
    df['cp'] != 4, 
    df['fbs'], 
    df['restecg'] == 0,
    df['exang'],
    df['restbp'] >= 130
]).T ## We transpose the matrix because numpy generates a matrix with selected rows not columns as we needed
y = np.array(df['hd'] > 0)

## Decision tree

Our goal is to write our own decision tree by implementing the greedy algorithm discussed on the lecture. 

Firstly, we want to compute Gini impurity for a given leaf and for a given node.

In [5]:
def gini_impurity_leaf(y):
    '''
    Find a value of gini impurity for a given leaf
    :param y: true binary labels correponds to this leaf
    :return gini: value of gini impurity
    '''
    # Do not forget that y can be empty! In this case use any value.
    if y.shape[0] == 0:
        return 0.5
    p1 = np.sum(y == 1) / y.shape[0]
    return 1 - (1-p1)**2 - p1**2

In [6]:
def gini_impurity_node(feature, y):
    '''
    Find a value of gini impurity for a given node that corresponds to the feature
    :param feature: binary feature vector corresponds to this node
    :param y: true binary labels correponds to this leaf
    :return gini:
    '''
    p = np.sum(feature == 1) / feature.shape[0]
    return p * gini_impurity_leaf(y[feature == 1]) + (1-p) * gini_impurity_leaf(y[feature == 0])

In [7]:
def find_best_feature(X,y):
    '''
    Find the best feature given dataset X and true labels y
    :param X: dataset matrix, rows corresponds to samples, colums to features
    :param y: true binary labels
    :return i: index that corresponds to feature column with least gini impurity
    :return gini: corresponding value of gini impurity
    '''
    best_feature = None
    best_gini = None
    for feature in range(X.shape[1]):
        gini = gini_impurity_node(X[:, feature], y)
        if best_gini is None or gini < best_gini:
            best_feature = feature
            best_gini = gini
    return best_feature, best_gini


Now we are ready to implement our own decision tree, using the functions above. We will use trees with fixed depth.

In [8]:
class DecisionTree:
    def __init__(self, max_depth):
        self._tree = {}
        self._max_depth = max_depth
    
    def fit(self, X, y):
        self._fit_node(X,y, self._tree, 0)
        return self
    
    def predict(self, X):
        predicted = []
        # We make predictions of each object
        for x in X:
            predicted.append(self._predict_node(x, self._tree))
        return np.array(predicted)
    
    def _fit_node(self, sub_X, sub_y, node, depth):
        if depth == self._max_depth or np.sum(sub_y == 1) == 0 or np.sum(sub_y == 0) == 0:
            # It is a leaf node! Use the most common value of sub_y here as a "value"
            node["type"] = "leaf"
            node["value"] = int(2 * np.sum(sub_y) > sub_y.shape[0])
            return
        # Otherwise it is an ordinary node. 
        node["type"] = "node"
        node["feature"] = find_best_feature(sub_X, sub_y)[0]
        node["left_child"] = {}
        node["right_child"] = {}
        # Recusive call to fill the rest of tree
        zero_mask = (sub_X[:, node["feature"]] == 0)
        one_mask = (sub_X[:, node["feature"]] == 1)
        self._fit_node(sub_X[zero_mask], sub_y[zero_mask], node["left_child"], depth+1)
        self._fit_node(sub_X[one_mask], sub_y[one_mask], node["right_child"], depth+1)
    
    def _predict_node(self, x, node):
        if node["type"] == "leaf":
            return node["value"]
        feature = node["feature"]
        # Left child corresponds to 0-value, right child corresponds to 1-value
        next_node = None
        if x[feature] == 0:
            next_node = node["left_child"]
        else:
            next_node = node["right_child"]
        return self._predict_node(x, next_node)

## Predict

Let us use this tree for classification of heart disease and test our predictions using sklearn library. We will use very simple tree of depth 2.

In [9]:
clf_dt = DecisionTree(max_depth=2)
clf_dt.fit(X,y)

Let us compute the accuracy on a given dataset:

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(clf_dt.predict(X), y)

ModuleNotFoundError: No module named 'sklearn'

One might argue that traing and testing on the same dataset is not fair. So, we need to split it to two different datasets and test it again:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf_dt = DecisionTree(max_depth=2)
clf_dt.fit(X_train, y_train)
accuracy_score(clf_dt.predict(X_test), y_test)

It shows us that the model does not overfit. The reason is clear -- the model is too simple and our features are too simple. Overfitting is pretty common for deep neural networks and huge decision trees. 

**Advice**: play around with the code. Playing with the code is the best way to learn from it.